In [ ]:
import ROOT
import glob
import math
import numpy as np
from IPython.display import display, Markdown
from IPython.display import display, HTML
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *

In [ ]:
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
# Read the Root Files
path = "/home/berki/alice/data/asyncQC/LHC22e/apass1/"
fileList = glob.glob(path+"*_QC.root")
fileList.sort()
runList = [fileList[i][55:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))
#rootDataFile[0].ls()
#rootDataFile[0].ClusterQC.ls()
#rootDataFile[0].PIDQC.ls()
#rootDataFile[0].TracksQC.ls()
#runList

# Reporting on TPC QC for LHC22e
> PilotBeam data from June 2022 <br />
> Link to JIRA: https://alice.its.cern.ch/jira/browse/O2-3035 <br />
> B = +0.5 T <br />
> beam = pp <br />
> energy (sqrt) = 900 GeV <br />
> Software version: O2PDPSuite::nightly-20220622-1, jq::v1.6-3 <br />
> Issues in FT0

## General comments
+ pT tail run dependance, appears to be correlated with filling scheme. To be investigated. Marking runs GOOD matched tracks don't seem to be affected.
+ More significant secondary bump for nCluster per track distribution compared to LHC22c

## Runs:
| Run number | Quality | Comment | 
| ---- | ---- | --- |
| **519041** | `GOOD` | |
| **519043** | `GOOD` | |
| **519045** | `GOOD` |  (Problem with one FEC on C-Side OROC1 (phi ~2.9)) |
| **519497** | `GOOD` | |
| **519498** | `GOOD` | |
| **519499** | `GOOD` | |
| **519502** | `GOOD` | |
| **519503** | `GOOD` | |
| **519504** | `GOOD` | |
| **519506** | `GOOD` | |
| **519507** | `GOOD` | |
| **519903** | `GOOD` | (Larger high-pT tail) |
| **519904** | `GOOD` | (Larger high-pT tail) |
| **519905** | `GOOD` | (Larger high-pT tail) |
| **519906** | `GOOD` | (Larger high-pT tail) |
| **519908** | `GOOD` | (Larger high-pT tail) |

---
## Number of track per run
+ Looking at hPt histogram from tracks task

In [ ]:
%jsroot on
objectName="hPt;1"
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,trend="entries",error="",log="logy",namesFromRunList=True)
canvas.Draw()

---
## Cluster QC


### NClusters

+ Increased noise on OROC3 of C4 and A10.
 + ==> Expected to be related to the powering of the field cage.

In [ ]:
%jsroot on
objectName = "c_N_Clusters"
c = rootDataFile[2].ClusterQC.Get(objectName)
c.Draw()

---
### QTot

+ One FEC on C-Side OROC1 had an issue during RUN 519045
 + Can e oserved in shown RUN QTot 519045 

In [ ]:
%jsroot on
objectName = "c_Q_Tot"
c = rootDataFile[2].ClusterQC.Get(objectName)
c.Draw()

---
### Sigma Time

- Increased noise on OROC3 of C4 and A10.
- Expected to be related to the powering of the field cage.

In [ ]:
%jsroot on
objectName = "c_Sigma_Time"
c = rootDataFile[2].ClusterQC.Get(objectName)
c.Draw()

---
## Tracks QC
**Notes:** Runs `OK`

## Phi distribution of tracks:

+ A-Side sector 5 and the A-side sector boundary between sectors 6-7 have significantly reduced tracks.
 + Couldn't correlate with observations from Cluster QC (bad FECs etc.)
 + Under investigation

In [ ]:
%jsroot on
objectName="hPhiAside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False, drawOption="SAME")

#Sector boundatries
pad1.cd()
sectorBoundary = ROOT.TLine(0,hist.GetMinimum(),0,hist.GetMaximum())
sectorBoundary.SetLineStyle(2)
for j in range(19):
    sectorBoundary.DrawLine((j*2*math.pi)/18,hist.GetMinimum(),(j*2*math.pi)/18,hist.GetMaximum())
    
canvas.Draw()

+ C-side no major issues observed

In [ ]:
%jsroot on
objectName="hPhiCside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False, drawOption="SAME L")
#Sector boundatries
pad1.cd()
sectorBoundary = ROOT.TLine(0,hist.GetMinimum(),0,hist.GetMaximum())
sectorBoundary.SetLineStyle(2)
for j in range(19):
    sectorBoundary.DrawLine((j*2*math.pi)/18,hist.GetMinimum(),(j*2*math.pi)/18,hist.GetMaximum())
canvas.Draw()

---
### Pt distribution

+ **Last 5 runs (`519903`,`519904`,`519905`,`519906`,`519908`,) have the increased high pT tail**
 + Was observed in runs `517753` & `517758` in LHC22c


In [ ]:
%jsroot on
objectName="hPt;1"
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,trend="mean",error="meanError",namesFromRunList=True)
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,log="logxy",legendNames=runList,pads=False)
canvas.Draw()

---
### Clusters per track

+ **Secondary peak at nClusters ~80 very significant**
 + Seen in all runs, no outliers
 + Similar peak was observed in MC (see report on LHC22c5) but not so prominent
 + Orignates mainly from A-side sector 7 and C-side sectors 2 & 12
 + In nClusters vs Eta peak is observed to be centered at eta=0 

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=False)
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile[0:1],normalize=False,legend=False,legendNames=runList[0:1],pads=True,drawOption="COLZ")
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile[0:1],normalize=False,legend=False,legendNames=runList[0:1],pads=True,drawOption="COLZ")
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile[-4:],normalize=False,legend=False,legendNames=runList[-4:],pads=True,drawOption="COLZ")
canvas.Draw()

---
### Sign of tracks

+ Ratio of pos/neg tracks consistent in all runs

In [ ]:
%jsroot on
objectName="hSign"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False)
canvas.Draw()

---
## PID QC

### MIP signal
+ No major outliers observed, all MIP peaks at ~50
 + All **GOOD**

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC"
quality = checkHistograms(objectName,rootDataFile,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",40,60)) and 45<histogram.GetFunction(\"gaus\").GetParameter(1)<55")
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,legendNames=runList,check=quality,normalize=False,log="logx",legend=False,pads=True)
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC;1"
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,trend="fit(gaus,Sq,,40,60)",error="",namesFromRunList=True,axis=1)
canvas.Draw()

---
### dEdx vs P
+ No unexpected shapes observed
+ All **GOOD**

In [ ]:
%jsroot on
objectName="hdEdxTotVsP_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile[0:1],log="logxz",pads=True,drawOption="COLZ")
canvas.Draw()